<a href="https://colab.research.google.com/github/gr-grey/genomic-courses/blob/main/naive_exact_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Exact Match

Find patterns that matches the given "lambda virus" genome sequence
https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa

Q1. How many times does AGGT or its reverse complement ACCT occur in the lambda virus genome? E.g. if AGGT occurs 10 times and ACCT occurs 12 times, you should report 22.

Q2. How many times does TTAA or its reverse complement occur in the lambda virus genome? Hint: TTAA and its reverse complement are equal, so remember not to double count.

Q3. What is the offset of the leftmost occurrence of ACTAAGT or its reverse complement in the Lambda virus genome? E.g. if the leftmost occurrence of ACTAAGT is at offset 40 (0-based) and the leftmost occurrence of its reverse complement ACTTAGT is at offset 29, then report 29.

Q4. What is the offset of the leftmost occurrence of AGTCGA or its reverse complement in the Lambda virus genome?

## Solution

We need to read the FastA file and store the entire genome as a string.

We need a naive exact match function that returns matching location of the patern p, given the sequence s.

To consider reverse complements, we need a function that turns a sequence string to its reverse complements, then get the reverse complements of both p and s.
If s does not equal to its reverse complement, we'll check the reverse complement matching and append the results.

In [ ]:
# download genome file
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa

--2023-04-07 19:32:02--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/lambda_virus.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.249.90.54, 13.249.90.71, 13.249.90.4, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.249.90.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49270 (48K) [application/octet-stream]
Saving to: ‘lambda_virus.fa.2’

lambda_virus.fa.2   100%[===================>]  48.12K  --.-KB/s    in 0.01s   

2023-04-07 19:32:02 (4.00 MB/s) - ‘lambda_virus.fa.2’ saved [49270/49270]



In [ ]:
# read FastA file and return the whole genome as a string
def readGenome(fastAfile):
    with open(fastAfile, 'r') as f:
        for line in f:
            if not line[0] == '>': # the line contains string
                # strip the endline char, add to genome
                genome += line.rstrip() 
    return genome

genome = readGenome('lambda_virus.fa')

In [ ]:
def naive(p,s): # exact match of p in s
    occurrences = []
    # check all possible locations in s
    for i in range(len(s) - len(p) + 1):
        match = True
        for j in range(len(p)): # check if substring matches
            if not p[j] == s[i+j]: # find mismatch; move on
                match = False
                break
        if match: # all chars matched; record
            occurrences.append(i)
    return occurrences
    
def reverseComplement(s):
    complement = {'A':'T', 'G':'C', 'C':'G', 'T':'A', 'N':'N'}
    rc_seq = ''
    for base in s:
        rc = complement[base]
        rc_seq = rc + rc_seq # prepend so the compelment is reversed
    return rc_seq

# navie with checking of reverse complement
def naive_rc(p, s): 
    occurrences = naive(p, s)
    p_rc = reverseComplement(p)
    # skip checking if p equals to its reverse complement
    if not p == p_rc:
        occurrences.extend(naive(p_rc, s))
    return occurrences 

In [ ]:
# Q1
print(len(naive_rc('AGGT', genome)))
# Q2
print(len(naive_rc('TTAA', genome)))
# Q3
print(min(naive_rc('ACTAAGT', genome)))
# Q4
print(min(naive_rc('AGTCGA', genome)))

306
195
26028
450


Q5. Make a new version of the naive function called naive_2mm that allows up to 2 mismatches per occurrence. Unlike for the previous questions, do not consider the reverse complement here. How many times does TTCAAGCC occur in the Lambda virus genome when allowing up to 2 mismatches?

Q6. What is the offset of the leftmost occurrence of AGGAGGTT in the Lambda virus genome when allowing up to 2 mismatches?

## Solution

To allow mismatches, we'll use a counter variable to record number of mismatches.
When mismatches happen, instead of immediately break out of the comparing loop, we check if the number of mismatches has exceeded the threshold, and only break when it does (in this case >2). 

In [ ]:
# naive matching with maximum of m errors
def naive_mm(p,s,m): 
    occurrences = []
    for i in range(len(s) - len(p) + 1):
        match = True
        mistakes = 0
        for j in range(len(p)):
            if not p[j] == s[i+j]: # find mismatch
                mistakes += 1
                if mistakes > m: # exceed error limits
                    match = False
                    break
        if match:
            occurrences.append(i)
    return occurrences

In [ ]:
# Q5
print(len(naive_mm('TTCAAGCC', genome, 2)))
# Q6
print(min(naive_mm('AGGAGGTT', genome, 2)))

191
49
